# Neighborhoods of Toronto

### Obtaied by scrapping the Wikipedia page and wrangle the data, clean it, and then read it into a pandas dataframe so that it is in a structured format 

In [53]:
import requests
import pandas as pd
import io

In [3]:
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text


In [4]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(website_url,'lxml')
#print(soup.prettify())

Obtain only the table  from the html data

In [5]:
My_table = soup.find('table',{'class':'wikitable sortable'})
#My_table

 The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

In [6]:
df = pd.read_html(str(My_table), header=0)
df = pd.DataFrame(df[0])

Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [7]:
df.drop(df[df.Borough == 'Not assigned'].index, inplace = True)

More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

In [8]:
df=df.groupby("Postcode").agg(lambda x:','.join(set(x)))
df.reset_index(inplace = True)

If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.

In [9]:
df.loc[df.Neighbourhood == 'Not assigned', 'Neighbourhood'] = df.loc[df.Neighbourhood == 'Not assigned', 'Borough']

In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [10]:
df.shape

(103, 3)

In [11]:
df.head(10)

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Rouge Hill,Highland Creek,Port Union"
2,M1E,Scarborough,"Morningside,Guildwood,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
7,M1L,Scarborough,"Oakridge,Clairlea,Golden Mile"
8,M1M,Scarborough,"Scarborough Village West,Cliffside,Cliffcrest"
9,M1N,Scarborough,"Cliffside West,Birch Cliff"


Get Geospatial data

In [86]:
url = 'http://cocl.us/Geospatial_data'
s = requests.get(url).content
geospatial_data = pd.read_csv(io.StringIO(s.decode('utf-8')))
geospatial_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Rename columns so that they match

In [110]:
df.columns = ['Postalcode', 'Borough', 'Neighbourhood']
geospatial_data.columns = ['Postalcode', 'Latitude', 'Longitude']

Merge both dataframes

In [111]:
neighborhood = pd.merge(df, geospatial_data, on = df['Postalcode'], right_index = True, left_index = True)
neighborhood.drop('Postalcode_y', axis =1, inplace = True)
neighborhood.columns= ['Postalcode', 'Borough', 'Neighbourhood', 'Latitude', 'Longitude']

In [118]:
neighborhood.head(10)

,Postalcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill,Highland Creek,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Morningside,Guildwood,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Oakridge,Clairlea,Golden Mile",43.711112,-79.284577
8,M1M,Scarborough,"Scarborough Village West,Cliffside,Cliffcrest",43.716316,-79.239476
9,M1N,Scarborough,"Cliffside West,Birch Cliff",43.692657,-79.264848
